In [0]:
%run "/Users/ovidiumtoma@gmail.com/wind_turbine_project/src/wt_ingestion"

In [0]:
# Initialize and trigger the ingestion class
ingestor = RawDataIngestor(spark)

# Define directory path where CSVs are stored
raw_directory_path = "dbfs:/FileStore/tables"

# Load both datasets into Bronze tables
df_original_turbine_data = ingestor.load_original_turbine_data(raw_directory_path)
ingestor.write_bronze(df_original_turbine_data, "original_turbine_bronze")

df_new_turbine_data = ingestor.load_new_turbine_data(raw_directory_path)
ingestor.write_bronze(df_new_turbine_data, "new_turbine_bronze")

print("Ingestion process completed.")

2025-02-21 22:35:44 - INFO - RawDataIngestor initialized.
2025-02-21 22:35:44 - INFO - Loading original turbine data from dbfs:/FileStore/tables
2025-02-21 22:35:46 - INFO - Successfully loaded 11160 original turbine dataset records.
2025-02-21 22:35:46 - INFO - Writing data to Delta table: bronze_data.original_turbine_bronze
2025-02-21 22:35:51 - INFO - Successfully written data to bronze_data.original_turbine_bronze
2025-02-21 22:35:51 - INFO - Loading new turbine data from dbfs:/FileStore/tables
2025-02-21 22:35:54 - INFO - Successfully loaded 175200 new turbine dataset records.
2025-02-21 22:35:54 - INFO - Maximum ID number from original dataset: 15
2025-02-21 22:35:54 - INFO - Successfully added turbine_id to new turbine dataset records.
2025-02-21 22:35:54 - INFO - Writing data to Delta table: bronze_data.new_turbine_bronze
2025-02-21 22:36:02 - INFO - Successfully written data to bronze_data.new_turbine_bronze
Ingestion process completed.
